# Automation Cytoscape network creation using RCy3 package
We are going to use the RCy3 package in R to automate the workflow for network creation.
We are using multiple gene epxression datasets acquired from different studies. These datasets contain gene expression profiles of patients with various diseases. 
The diseases which are going to be used in this notebook are: lung-cancer (LC), breast-cancer (BC), metabolically unhealthy obesity (MUO), rheumatoid arthritis (RA), Rett syndrome frontal cortex (Rett_FC), Rett syndrome temporal cortex (Rett_TC) and systemis lupus erythematosus (SLE).

The network will consist of pathways, pathway clusters and the genes which occur in these pathways. These pathways are acquired based on genes associated with inflammation which were retrieved from [DisGeNET](http://www.disgenet.org/home/) and [GeneCards](https://www.genecards.org/). 

### The following step only works in RStudio. If working in another environment, please set the working directory properly and check if the working directory is correct. 

In [1]:
# set wd to where script file is saved
setwd(dirname(rstudioapi::callFun("getActiveDocumentContext")$path))

ERROR: Error: RStudio not running


In [2]:
# check wd
getwd()

[1] "C:/Users/Laurent/Documents/GitHub/inflammation_networks2/Jupyter - RCy3 network creation"

In [3]:
# load library
library(RCy3)

### We first start with opening cytoscape and checking if we are connected

In [4]:
# check if cytoscape is open and check version
cytoscapePing()
cytoscapeVersionInfo()

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): Failed to connect to localhost port 1234: Connection refused


### Network creation
To begin with, we will load in our node and edge files to create our network.

In [10]:
# load in data of network
nodes <- as.data.frame(read.table(file.path(getwd(), "Network", "nodes_entrezgene.txt"), header = T, sep = "\t", stringsAsFactors = FALSE))
edges <- as.data.frame(read.table(file.path(getwd(), "Network", "edges_entrezgene.txt"), header = T, sep = "\t", stringsAsFactors = FALSE))

We have to clean these files so they are usable for network creation.

In [11]:
# clean up data
colnames(nodes)[1] <- "id"
nodes$id <- as.character(nodes$id)
edges <- edges[,c(-3,-4)]
colnames(edges)[c(1,2)] <- c("source", "target") 
edges$interaction <- "interacts"
edges$target <- as.character(edges$target)

head(nodes)
head(edges)

id,Type
WP_OVERVIEW_OF_NANOPARTICLE_EFFECTS,Process
WP_FIBRIN_COMPLEMENT_RECEPTOR_3_SIGNALING_PATHWAY,Process
WP_DEVELOPMENT_AND_HETEROGENEITY_OF_THE_ILC_FAMILY,Process
Vitamin_B12,Process
NFkB,Process
Inflammation,Process


source,target,interaction
Inflammation,10000,interacts
Inflammation,10000,interacts
WP_OVERVIEW_OF_NANOPARTICLE_EFFECTS,10000,interacts
WP_OVERVIEW_OF_NANOPARTICLE_EFFECTS,1001,interacts
Vitamin_B12,100129518,interacts
Vitamin_B12,100129518,interacts


Now we can create our network in Cytoscape

In [12]:
# create network from the data frames
createNetworkFromDataFrames(nodes, edges, title = "MyNetwork", collection = "MyCollection")

Loading data...
Applying default style...
Applying preferred layout...


networkSUID 
       2352

We have to load in our gene expression data file, and merge this file with the table in cytoscape.

In [13]:
# load data set with gene expression values (logFC, p.value)
expr_data <- read.table(file.path(getwd(), "expr_data", "merged_data_final.txt"), header = T, sep ="\t")

# load data into cytoscape
loadTableData(expr_data, data.key.colum = "entrezgene", table.key.column = "shared name")

# check if tables are well merged
nodeTable <- getTableColumns(table = "node")
head(nodeTable)

[1] "Success: Data loaded in defaultnode table"

,SUID,shared name,id,type,Entrez Gene,name,selected,entrezgene,hgnc_symbol,logFC_BC,...,logFC_MUO,PValue_MUO,logFC_RA,PValue_RA,logFC_RETT_FC,PValue_RETT_FC,logFC_RETT_TC,PValue_RETT_TC,logFC_SLE,PValue_SLE
2362,2362,WP_OVERVIEW_OF_NANOPARTICLE_EFFECTS,WP_OVERVIEW_OF_NANOPARTICLE_EFFECTS,Process,NA,WP_OVERVIEW_OF_NANOPARTICLE_EFFECTS,FALSE,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2363,2363,WP_FIBRIN_COMPLEMENT_RECEPTOR_3_SIGNALING_PATHWAY,WP_FIBRIN_COMPLEMENT_RECEPTOR_3_SIGNALING_PATHWAY,Process,NA,WP_FIBRIN_COMPLEMENT_RECEPTOR_3_SIGNALING_PATHWAY,FALSE,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2364,2364,WP_DEVELOPMENT_AND_HETEROGENEITY_OF_THE_ILC_FAMILY,WP_DEVELOPMENT_AND_HETEROGENEITY_OF_THE_ILC_FAMILY,Process,NA,WP_DEVELOPMENT_AND_HETEROGENEITY_OF_THE_ILC_FAMILY,FALSE,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2365,2365,Vitamin_B12,Vitamin_B12,Process,NA,Vitamin_B12,FALSE,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2366,2366,NFkB,NFkB,Process,NA,NFkB,FALSE,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2367,2367,Inflammation,Inflammation,Process,NA,Inflammation,FALSE,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Visualization 
Before we start to visualize our network, we have to perform some manual stuff! 

In [14]:
# first we will get rid off the manual stuff
system('CMD /C "ECHO To analyze the network we have to go to Cytoscape. First Open cytoscape go to Tools then NetworkAnalyzer then Network Analysis then Analyze Network. Treat the network as unidirected and click OK. We analyzed the network now we want to change the layout of the network. Go to Layout then yFiles Organic Layout. Close this message box when finished, and return to R. && PAUSE"', 
       invisible=FALSE, wait=FALSE)

Now everthing is loaded into cytoscape and the manual stuff is done, we can working on the automatization of the visual style.

In [15]:
# lock width and height of nodes
lockNodeDimensions(TRUE)

For every dataset we would like to create a own network. We will do this using the clone network function.

In [16]:
# we have 7 gene expression sets so we repeat this function 8 times, we will use a repeat loop for this
diseases <- c("BC","LC","MUO","RA","RETT_FC","RETT_TC","SLE")

x <- 1
repeat {
  cloneNetwork(network = "MyNetwork")
  setCurrentNetwork(network="MyNetwork_1")
  renameNetwork(diseases[x])
  x = x + 1
  if (x == length(diseases)+1){
  print("Done!")
  break}
}

[1] "Done!"


Now we have a network available for every dataset we have, we can set a visual style which maps the gene expression (logFC) and significance (p-value) for every gene node in the network per dataset.

In [18]:
# Map logFC and pValue per disease for every respective network and add visual style to every network
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  
  style.name = paste("style_", diseases[x], sep = "")
  
  # map logFC of gene expression data per disease on respective networks
  mappings <- list(nodeFill <- mapVisualProperty("node fill color", table.column = paste("logFC_", diseases[x] , sep = ""), 
                                    mapping.type = "continuous",
                                    c(-0.58,0 ,0.58),
                                    c("#0000FF", "#FFFFFF", "#FF0000")),
  
  # map significance per disease on respective networks
  nodeBorder <- mapVisualProperty("node border paint", table.column = paste("PValue_", diseases[x], sep = ""),
                                    mapping.type = "continuous",
                                    table.column.values = c(0.00, 0.05, 1.00), 
                                    c("#00FF00", "#FFFFFF", "#FFFFFF")),
  
  # map node shapes
  nodeShape <- mapVisualProperty("Node Shape", table.column = "type", 
                                    mapping.type = "discrete", 
                                    table.column.values = c("Gene", "Pathway", "Cluster", "InflGene"),
                                    c("DIAMOND", "ELLIPSE", "ROUND_RECTANGLE", "VEE")),
  
  label <- mapVisualProperty("Node Label", table.column = "shared name",
                             mapping.type = "passthrough"))

  defaults <- list(NODE_FILL_COLOR = "#999999",
                   NODE_BORDER_PAINT = "#999999",
                   NODE_BORDER_WIDTH = 7,
                   NODE_LABEL_FONT_SIZE = 18,
                   NETWORK_TITLE = diseases[x])
  
  # create and set unique visual style per disease dataset
  createVisualStyle(style.name, defaults, mappings = mappings)
  setVisualStyle(style.name)

  x = x + 1
  if (x == length(diseases)+1){
    print("Done!")
    break}
}

[1] "Done!"


Before we are going to select nodes (which we cant deselect...), we are going to save images of the main networks. We do this before selecting nodes so that our images don't contain the yellow highlighted selected nodes. Prior to this step, make sure there are no images saved with the same name, otherwise this part will not work!

In [19]:
# save disease networks
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  fitContent()
  
  png.file <- file.path(getwd(), "Images", "networks", paste("gene_expr_network", diseases[x], ".png", sep = ""))
  exportImage(png.file, type = "png", resolution=600, zoom=500)
  
  x = x + 1
  if (x == length(diseases)+1){
    print("Done!")
    break}
}

[1] "Done!"


### Subnetwork creation
We are also interested in the significant differentially expressed genes and inflammation associated genes. So lets make subnetworks out of these properties!

In [21]:
# create subnetwork of significant differentially expressed genes
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  
  createColumnFilter(filter.name = "expr genes",     column = paste("logFC_", diseases[x], sep = "") , c(-0.58,0.58), "IS_NOT_BETWEEN")
  createColumnFilter(filter.name = "sig genes",      column = paste("PValue_", diseases[x], sep = ""), 0.05, "LESS_THAN")
  sigexpr <- createCompositeFilter('combined filter', filter.list = c("sig genes", "expr genes"), type = "ALL")
  
  process <- createColumnFilter(filter.name = "Pathway filter", column = "type", "Process", "IS")
  #clusters <- createColumnFilter(filter.name = "Cluster filter", column = "Type", "Cluster", "IS")
  
  selectNodes(nodes = c(process$nodes, sigexpr$nodes), by.col = "shared name")
  
  createSubnetwork(nodes = "selected", subnetwork.name = paste("subnetwork ", diseases[x], " genes", sep = ""))
  
  x = x + 1
  if (x == length(diseases)+1){
    print("Done!")
    break}
}

[1] "Done!"


In [22]:
# create subnetwork of significant differentially expressed inflammation genes
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  
  createColumnFilter(filter.name = "Infl filter",    column = "type", "InflGene", "IS")
  createColumnFilter(filter.name = "expr genes",     column = paste("logFC_", diseases[x], sep = "") , c(-0.58,0.58), "IS_NOT_BETWEEN")
  createColumnFilter(filter.name = "sig genes",      column = paste("PValue_", diseases[x], sep = ""), 0.05, "LESS_THAN")
  sigexpr <- createCompositeFilter('combined filter', filter.list = c("Infl filter", "sig genes", "expr genes"), type = "ALL")
  
    process <- createColumnFilter(filter.name = "Pathway filter", column = "type", "Process", "IS")
  #clusters <- createColumnFilter(filter.name = "Cluster filter", column = "Type", "Cluster", "IS")
  
  selectNodes(nodes = c(process$nodes, sigexpr$nodes), by.col = "shared name")
  
  createSubnetwork(nodes = "selected", subnetwork.name = paste("subnetwork ", diseases[x], " inflGenes",sep = ""))
  
  x = x + 1
  if (x == length(diseases)+1){
    print("Done!")
    break}
}

[1] "Done!"


### Saving everything
We are almost finished! We already saved the images of the main networks for every disease, but we still want to save the images of the subnetworks. So lets do that, and after we will save the cytoscape session so nothing is lost.

In [23]:
# save subnetworks genes
subgenes <- c("subnetwork BC genes","subnetwork LC genes","subnetwork MUO genes",
              "subnetwork RA genes","subnetwork RETT_FC genes",
              "subnetwork RETT_TC genes","subnetwork SLE genes")
x <- 1
repeat {
  setCurrentNetwork(network = subgenes[x])
  fitContent()
  
  png.file <- file.path(getwd(), "Images", "subGeneNetworks", paste("subgene_expr_network", diseases[x], ".png", sep = ""))
  exportImage(png.file, type = "png", resolution=600, zoom=500)
  
  x = x + 1
  if (x == length(diseases)+1){
    print("Done!")
    break}
}

[1] "Done!"


In [24]:
# save subnetworks inflammation genes
subInflGenes <- c("subnetwork BC inflGenes","subnetwork LC inflGenes","subnetwork MUO inflGenes",
              "subnetwork RA inflGenes","subnetwork RETT_FC inflGenes",
              "subnetwork RETT_TC inflGenes","subnetwork SLE inflGenes")
x <- 1
repeat {
  setCurrentNetwork(network = subInflGenes[x])
  fitContent()
  
  png.file <- file.path(getwd(), "Images", "subInflGeneNetworks", paste("subinflgene_expr_network", diseases[x], ".png", sep = ""))
  exportImage(png.file, type = "png", resolution=600, zoom=500)
  
  x = x + 1
  if (x == length(diseases)+1){
    print("Done!")
    break}
}

[1] "Done!"


In [25]:
# save Cytoscape session file
session.file <- file.path(getwd(), "Network", "gene_expr_networks.cys")
saveSession(session.file)